In [1]:
# libraries
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [2]:
window = tk.Tk() # parent widget
window.geometry("1080x640")
window.wm_title("Trading")

''

### 1-Main Paned Window (pw_parent): contains Left Side and Right Side.

#### Note
Solda **altlı üstlü(vertical)** iki adet **frame**'i kapsayan bir adet **paned window (pw1)**, sağ tarafta da bir adet **frame** olacağı için, **horizontal** olacak şekilde, hem sol hem sağ tarafı kapsayan bir **paned window(pw_parent)** oluşturuldu.

In [3]:
pw_parent = ttk.PanedWindow(window, orient = tk.HORIZONTAL)
pw_parent.pack(fill = tk.BOTH, expand = True)

In [4]:
pw1 = ttk.PanedWindow(pw_parent, orient = tk.VERTICAL)

In [5]:
# relief = tk.SUNKEN => kenarların nasıl görüneceği ile ilgili bir görsellik.
frame1 = ttk.Frame(pw_parent, width = 360, height = 640, relief = tk.SUNKEN)
frame2 = ttk.Frame(pw_parent, width = 720, height = 400, relief = tk.SUNKEN)
frame3 = ttk.Frame(pw_parent, width = 720, height = 240, relief = tk.SUNKEN)

In [6]:
pw1.add(frame2)
pw1.add(frame3)

In [7]:
pw_parent.add(pw1)
pw_parent.add(frame1)

### 2-Right Side(**frame1**) : contains a **i. treeview** and a **ii. button**.

#### i. treeview

#### Note
**item** değişkenini başka fonksiyonlar içinde de kullanabilmek için **global** olarak tanımlarız.

In [8]:
item = ""
def callback(event):
    global item 
    item = treeview.identify("item", event.x,event.y)
    #print("Clicked: ", item)

In [9]:
treeview = ttk.Treeview(frame1)
treeview.grid(row = 0, column = 1, padx = 25 , pady = 25)
treeview.insert("", "0", "Major", text = "Major")
treeview.insert("Major", "1", "EUR/USD", text ="EUR/USD") #USD varsa Major'un altına gelecek.
treeview.insert("", "2", "Minor", text = "Minor")
treeview.insert("Minor", "3", "EUR/GBR", text ="EUR/GBR") #USD yoksa Minor'un altına gelecek.
treeview.bind("<Double-1>", callback)

'2611635679296callback'

#### ii. button

#### Note
We need use a **raw string** (**r**) for the path variable in reading **txt** files.

In [10]:
def readNews(item):
    if item == "EUR/USD":
        news = pd.read_csv(r"data\news_eur_usd.txt")
    elif item == "EUR/GBR":
        news = pd.read_csv(r"data\news_eur_gbr.txt")
    textBox.insert(tk.INSERT, news)

In [11]:
def openTrade():
    global data, future, line, canvas, data_close_array, future_array, ax1,line2,canvas2,ax2,line3,canvas3,ax3,line4,canvas4,ax4
    print('openTrade')
    if item != "":
        print("Chosen item: ", item)
        if item == "EUR/USD":
            # button setting
            open_button.config(state = "disabled")
            start_button.config(state = "normal")
            
            # read data
            data = pd.read_csv("data\eurusd.csv")

            # split data
            future = data[-1000:] # son 1000 tanesini similasyon olarak devamlı eklenecek şekilde gösterilecek.
            data = data[:len(data)-1000]
            data_close_array = data.close1.values # kur'un kapanış değeri
            future_array = list(future.close1.values)

            # line plot
            fig1 = plt.Figure(figsize =(5,4), dpi = 100)
            ax1 = fig1.add_subplot(111) #1. satır, 1. sütun, 1 subplot
            # 2 adet variable dönüyor, birini line yaptık, diğeri önemsiz (,)
            line, = ax1.plot(range(len(data)), data.close1,color = "blue")
            
            canvas = FigureCanvasTkAgg(fig1, master = tab1)
            canvas.draw()
            canvas.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = 1)

            # scatter
            # 1 adet variable dönüyor, o yüzden line2 demek yeterli
            fig2 = plt.Figure(figsize =(5,4), dpi = 100)
            ax2 = fig2.add_subplot(111)
            line2 = ax2.scatter(range(len(data)), data.close1,s = 1, alpha = 0.5, color = "blue") # s=size=1, opacity=alpha=0.5
            
            canvas2 = FigureCanvasTkAgg(fig2, master = tab2)
            canvas2.draw()
            canvas2.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = 1)

            # read news about EUR/USD
            readNews(item)

        elif item == "EUR/GBR":
            
            # button setting
            open_button.config(state = "disabled")
            start_button.config(state = "normal")
            
            # read data
            data = pd.read_csv("data\eurgbr.csv")
            
            # split data
            future = data[-1000:]
            data = data[:len(data)-1000]
            data_close_array = data.close1.values
            future_array = list(future.close1.values)
            
            # line
            fig3 = plt.Figure(figsize =(5,4), dpi = 100)
            ax3 = fig3.add_subplot(111)
            line3, = ax3.plot(range(len(data)),data.close1,color = "blue")
            
            canvas3 = FigureCanvasTkAgg(fig3, master = tab1)
            canvas3.draw()
            canvas3.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = 1)

            # scatter
            fig4 = plt.Figure(figsize =(5,4), dpi = 100)
            ax4 = fig4.add_subplot(111)
            line4 = ax4.scatter(range(len(data)),data.close1,s = 1, alpha = 0.5, color = "blue")
            
            canvas4 = FigureCanvasTkAgg(fig4, master = tab2)
            canvas4.draw()
            canvas4.get_tk_widget().pack(side = tk.TOP, fill = tk.BOTH, expand = 1)

            # read news about EUR/GBR
            readNews(item)

        else: # seçilen item EUR/USD ya da EUR/GBR değilse hata versin
            messagebox.showinfo(title = "Warning", message = "Double click to choose currency pair")
    else: # hiçbir şey seçmeden open trading butonuna basılırsa, alert versin.
        messagebox.showinfo(title = "Warning", message = "Double click to choose currency pair") 

In [12]:
open_button = tk.Button(frame1, text = "Open Trading", command = openTrade)
open_button.grid(row = 2, column = 1, padx = 5, pady = 5)

### 3-Left Side(**pw1**) contains **frame2** and **frame3**
#### A) **frame3**: contains **text editor** and **scroll bar**

In [13]:
textBox = tk.Text(frame3, width = 70, height = 10, wrap = "word") # contains word(string)
textBox.grid(row = 0, column = 0, padx =25, pady = 25)

In [14]:
scroll = tk.Scrollbar(frame3, orient = tk.VERTICAL, command = textBox.yview)
scroll.grid(row = 0, column = 1, sticky = tk.N + tk.S, pady=10)
textBox.config(yscrollcommand = scroll.set) # textbox'a scroll özelliği ekledik.

#### B) **frame2**: contains **tabs**, **radio button**, **button**, **result(labelframe)**, **plot**

In [15]:
# tabs
tabs = ttk.Notebook(frame2, width = 540, height = 300)
tabs.place(x = 25, y= 25)

tab1 = ttk.Frame(tabs, width = 50, height = 50)
tab2 = ttk.Frame(tabs)

tabs.add(tab1, text = "Line")
tabs.add(tab2, text = "Scatter", compound = tk.LEFT)

In [16]:
# radio button
method = tk.StringVar()
tk.Radiobutton(frame2, text = "m1: ", value = "m1", variable = method).place(x = 580, y= 100)
tk.Radiobutton(frame2, text = "m2: ", value = "m2", variable = method).place(x = 580, y= 125)

In [17]:
# label frame: result
label_frame = tk.LabelFrame(frame2, text = "Result", width = 100, height = 150)
label_frame.place(x = 580, y = 25)
tk.Label(label_frame, text = "Buy: ", bd = 3).grid(row = 0, column = 0)
tk.Label(label_frame, text = "Sell: ", bd = 3).grid(row = 1, column = 0)

In [18]:
# buy sell labels 
# aşağıdaki value'lar plot update edildikçe güncellenecek.
buy_value = tk.Label(label_frame, text = "1", bd = 3)
buy_value.grid(row = 0, column = 1)
sell_value = tk.Label(label_frame, text = "0", bd = 3)
sell_value.grid(row = 1, column = 1)

#### cumsum nedir?
```python
a = np.array([1,2,3,4])
result = np.cumsum(a)

print(result)
[ 1  3  6 10] 
```

In [19]:
def moving_average(a, n = 50):
    result = np.cumsum(a, dtype= float) # kümülatif toplama işlemi
    # n. indexten sonraki değerleri, n. indexe kadar olan değerlerinden çıkartıyorum(moving operation)
    result[n:] = result[n:] - result[:-n]
    return result[n-1:]/n # n-1 indexten sonuna kadar git ve n'e böl. (Average)

In [20]:
def update():
    global data_close_array, ax1,ax2,ax3,ax4
    
    spread = 0.0002 # buy ve sell value'ları farklı olsun diye aralarında küçük bir spread farkı oluşturuldu.
    
    # data_close_array inin en son elemanı olsun ve spread çıkarılsın ve virgülden sonra 5 basamağı göster.
    buy_value.config(text = str((data_close_array[-1]-spread).round(5)))
    sell_value.config(text = str((data_close_array[-1]+spread).round(5)))
    
    window.after(599, update) # 500 milisaniyede bir data_close_array güncellenecek.
    
    # değerleri ekranda güncellemek için:
    # future_array'in ilk elemanını, dataclosearray'in en sonuna append(ekleme) yaparız.
    data_close_array = np.append(data_close_array,future_array.pop(0))
    
    if method.get() == "m1": # metot1
        if item == "EUR/USD":
            # line
            ax1.set_xlim(0,len(data_close_array) + 10) # eklenen verinin sınırını genişletmek gerekir.
            line.set_ydata(data_close_array)
            line.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax2.set_xlim(0,len(data_close_array) + 10)
            ax2.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            # moving average
            n = 50
            mid_rolling = moving_average(data_close_array,n)
            ax1.plot(range(n-1,len(data_close_array)), mid_rolling, linestyle = "--", color = "red")
            ax2.plot(range(n-1,len(data_close_array)) ,mid_rolling, linestyle = "--", color = "red")
            
            canvas.draw()
            canvas2.draw()
        
        elif item == "EUR/GBR":
            # line
            ax3.set_xlim(0,len(data_close_array) + 10)
            line3.set_ydata(data_close_array)
            line3.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax4.set_xlim(0,len(data_close_array) + 10)
            ax4.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            # moving average
            n = 50
            mid_rolling = moving_average(data_close_array,n)
            ax3.plot(range(n-1,len(data_close_array)),mid_rolling,linestyle = "--", color = "red")
            ax4.plot(range(n-1,len(data_close_array)),mid_rolling,linestyle = "--", color = "red")
            
            canvas3.draw()
            canvas4.draw()
            
    elif method.get() == "m2":
        if item == "EUR/USD":
            # line
            ax1.set_xlim(0,len(data_close_array) + 10)
            line.set_ydata(data_close_array)
            line.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax2.set_xlim(0,len(data_close_array) + 10)
            ax2.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            # moving average
            n = 200
            long_rolling = moving_average(data_close_array,n)
            ax1.plot(range(n-1,len(data_close_array)),long_rolling,linestyle = "--", color = "green")
            ax2.plot(range(n-1,len(data_close_array)),long_rolling,linestyle = "--", color = "green")
            
            canvas.draw()
            canvas2.draw()
        
        elif item == "EUR/GBR":
            # line
            ax3.set_xlim(0,len(data_close_array) + 10)
            line3.set_ydata(data_close_array)
            line3.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax4.set_xlim(0,len(data_close_array) + 10)
            ax4.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            # moving average
            n = 200
            long_rolling = moving_average(data_close_array,n)
            ax3.plot(range(n-1,len(data_close_array)),long_rolling,linestyle = "--", color = "green")
            ax4.plot(range(n-1,len(data_close_array)),long_rolling,linestyle = "--", color = "green")
            
            canvas3.draw()
            canvas4.draw()
    else:
        if item == "EUR/USD":
            # line
            ax1.set_xlim(0,len(data_close_array) + 10)
            line.set_ydata(data_close_array)
            line.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax2.set_xlim(0,len(data_close_array) + 10)
            ax2.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            canvas.draw()
            canvas2.draw()
        elif item == "EUR/GBR":
            # line
            ax3.set_xlim(0,len(data_close_array) + 10)
            line3.set_ydata(data_close_array)
            line3.set_xdata(range(len(data_close_array)))
            
            # scatter
            ax4.set_xlim(0,len(data_close_array) + 10)
            ax4.scatter(range(len(data_close_array)), data_close_array, s = 1, alpha = 0.5, color = "blue")
            
            canvas3.draw()
            canvas4.draw()

In [21]:
# button
def startTrading():
    # after(time(ms), function) = Call the specified function after the given time.
    # 0 = butona basılır basılmaz hemen update function'una git anlamına geliyor.
    window.after(0, update)
    print("startTrading")
start_button = tk.Button(frame2, text = "Start Trading", command = startTrading)
start_button.place(x=580, y = 150)
start_button.config(state = "disabled")


In [22]:
window.mainloop()

openTrade
Chosen item:  EUR/USD
startTrading
